# Dynamic Pricing / Discount Optimization Model

## Farm2Home ML Model Training

This notebook trains a machine learning model to predict optimal discounts (0-30%) for products that maximize revenue while maintaining conversion rates.

**Features:**
- Product base price
- Category
- Past sales volume
- Discount percentage
- Seasonal features (month, day of week)
- Price elasticity estimation

**Output:** 
- `dynamic_pricing_model.pkl` - Trained model
- `input_scaler.pkl` - Feature scaler
- `product_encoder.pkl` - Category encoder

**Goal:** Suggest optimal discount % (0-30%) that maximizes revenue per product.


## 1. Import Libraries


In [39]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
import xgboost as xgb
import os

warnings.filterwarnings('ignore')

print("✅ Libraries imported successfully")


✅ Libraries imported successfully


## 2. Load Order History Dataset


In [40]:
# Load data from CSV files or generate synthetic data
print("📚 Loading order history dataset...")

try:
    # Try loading from CSV files
    order_history_df = pd.read_csv('../order_history.csv')
    print("✅ Loaded from CSV file")
except:
    try:
        order_history_df = pd.read_csv('order_history.csv')
        print("✅ Loaded from current directory")
    except Exception as e:
        print(f"⚠️  Could not load CSV files: {e}")
        print("📊 Generating synthetic order history data for training...")
        
        # Generate comprehensive synthetic data
        np.random.seed(42)
        n_records = 5000
        
        # Product categories
        categories = ['Vegetables', 'Fruits', 'Grains', 'Dairy', 'Spices', 'Pulses', 'Oil', 'Nuts']
        
        # Product base prices (varies by category)
        category_base_prices = {
            'Vegetables': (50, 200),
            'Fruits': (80, 300),
            'Grains': (40, 150),
            'Dairy': (60, 250),
            'Spices': (100, 500),
            'Pulses': (70, 200),
            'Oil': (150, 400),
            'Nuts': (200, 800)
        }
        
        # Generate synthetic data
        data = []
        products = {}
        
        for i in range(n_records):
            category = np.random.choice(categories)
            product_id = f"PROD_{category}_{np.random.randint(1, 100)}"
            
            # Base price based on category
            min_price, max_price = category_base_prices[category]
            base_price = np.random.uniform(min_price, max_price)
            
            # Discount percentage (0-30%)
            offered_discount = np.random.uniform(0, 30)
            
            # Discount affects quantity ordered (price elasticity)
            # Higher discount = more quantity, but with diminishing returns
            elasticity_factor = 1 + (offered_discount / 100) * np.random.uniform(0.5, 2.0)
            base_quantity = np.random.randint(1, 20)
            quantity_ordered = max(1, int(base_quantity * elasticity_factor))
            
            # Conversion rate (higher discount = higher conversion, but not linear)
            base_conversion = np.random.uniform(0.3, 0.9)
            conversion_boost = (offered_discount / 100) * np.random.uniform(0.1, 0.3)
            conversion_rate = min(1.0, base_conversion + conversion_boost)
            
            # Purchase decision (binary)
            purchased = 1 if np.random.random() < conversion_rate else 0
            
            # Past sales volume (cumulative)
            if product_id not in products:
                products[product_id] = {
                    'past_sales_volume': 0,
                    'product_name': f"{category} Product {len([p for p in products.keys() if category in p]) + 1}"
                }
            
            # Date feature (seasonality)
            days_ago = np.random.randint(0, 365)
            order_date = datetime.now() - timedelta(days=days_ago)
            
            # Update past sales volume
            if purchased:
                products[product_id]['past_sales_volume'] += quantity_ordered
            
            data.append({
                'product_id': product_id,
                'product_name': products[product_id]['product_name'],
                'base_price': round(base_price, 2),
                'offered_discount': round(offered_discount, 2),
                'quantity_ordered': quantity_ordered if purchased else 0,
                'conversion_rate': round(conversion_rate, 4),
                'purchased': purchased,
                'category': category,
                'past_sales_volume': products[product_id]['past_sales_volume'],
                'order_date': order_date,
                'month': order_date.month,
                'day_of_week': order_date.weekday(),
                'is_weekend': 1 if order_date.weekday() >= 5 else 0
            })
        
        order_history_df = pd.DataFrame(data)
        print(f"✅ Generated {len(order_history_df)} synthetic records")
        print(f"📊 Records with purchases: {order_history_df['purchased'].sum()}")
        print(f"📊 Unique products: {order_history_df['product_id'].nunique()}")
        print(f"📊 Categories: {order_history_df['category'].nunique()}")

print("\n📋 Dataset Info:")
print(order_history_df.head())
print(f"\n📊 Dataset shape: {order_history_df.shape}")
print(f"📊 Missing values:\n{order_history_df.isnull().sum()}")


📚 Loading order history dataset...
⚠️  Could not load CSV files: [Errno 2] No such file or directory: 'order_history.csv'
📊 Generating synthetic order history data for training...
✅ Generated 5000 synthetic records
📊 Records with purchases: 3138
📊 Unique products: 791
📊 Categories: 8

📋 Dataset Info:
       product_id      product_name  base_price  offered_discount  \
0     PROD_Oil_52     Oil Product 1      387.68             21.96   
1    PROD_Nuts_24    Nuts Product 1      590.53              1.69   
2  PROD_Pulses_89  Pulses Product 1      107.86             18.36   
3     PROD_Oil_64     Oil Product 2      266.69             25.80   
4   PROD_Fruits_9  Fruits Product 1       83.51              6.93   

   quantity_ordered  conversion_rate  purchased category  past_sales_volume  \
0                 9           0.5939          1      Oil                  9   
1                 0           0.3055          0     Nuts                  0   
2                 0           0.4613          

## 3. Feature Engineering


In [41]:
print("🔧 Engineering features...")

# Create a copy for feature engineering
df = order_history_df.copy()

# 1. Discounted price
df['discounted_price'] = df['base_price'] * (1 - df['offered_discount'] / 100)

# 2. Revenue (only for purchased items)
df['revenue'] = df['discounted_price'] * df['quantity_ordered'] * df['purchased']

# 3. Price elasticity estimate
# Price elasticity = % change in quantity / % change in price
# Higher discount should increase quantity, but effect diminishes
df['price_change_pct'] = df['offered_discount'] / 100
df['quantity_change_pct'] = np.where(
    df['purchased'] == 1,
    (df['quantity_ordered'] - df.groupby('product_id')['quantity_ordered'].transform('mean')) / 
    (df.groupby('product_id')['quantity_ordered'].transform('mean') + 1e-6),
    0
)

# Simplified elasticity estimate
df['price_elasticity_estimate'] = np.where(
    df['price_change_pct'] > 0,
    df['quantity_change_pct'] / (df['price_change_pct'] + 1e-6),
    -1.0  # Default for no discount
)

# Clip extreme values
df['price_elasticity_estimate'] = df['price_elasticity_estimate'].clip(-5, 5)

# 4. Discount category (bins)
df['discount_category'] = pd.cut(
    df['offered_discount'],
    bins=[-1, 5, 10, 15, 20, 25, 30],
    labels=['0-5%', '5-10%', '10-15%', '15-20%', '20-25%', '25-30%']
)

# 5. Revenue per unit
df['revenue_per_unit'] = df['revenue'] / (df['quantity_ordered'] + 1e-6)

# 6. Profit margin estimate (assuming 30% margin on base price)
df['cost_per_unit'] = df['base_price'] * 0.7
df['profit_per_unit'] = df['discounted_price'] - df['cost_per_unit']
df['total_profit'] = df['profit_per_unit'] * df['quantity_ordered'] * df['purchased']

# 7. Seasonal features (already extracted in data generation)
# month, day_of_week, is_weekend

# 8. Category encoding (will be done later)
# category

# 9. Past performance features
df['avg_past_sales'] = df.groupby('product_id')['past_sales_volume'].transform('mean')
df['sales_volume_percentile'] = df.groupby('category')['past_sales_volume'].transform(
    lambda x: pd.qcut(x, q=4, labels=False, duplicates='drop')
).fillna(0)

print("✅ Feature engineering completed")
print(f"\n📋 Engineered features:\n{df[['discounted_price', 'revenue', 'price_elasticity_estimate', 'total_profit']].head()}")
print(f"\n📊 Revenue statistics:")
print(df[df['purchased'] == 1]['revenue'].describe())


🔧 Engineering features...
✅ Feature engineering completed

📋 Engineered features:
   discounted_price      revenue  price_elasticity_estimate  total_profit
0        302.545472  2722.909248                   4.307566    280.525248
1        580.550043     0.000000                   0.000000      0.000000
2         88.056904     0.000000                   0.000000      0.000000
3        197.883980     0.000000                   0.000000      0.000000
4         77.722757   544.059299                   5.000000    134.860299

📊 Revenue statistics:
count     3138.000000
mean      2152.531580
std       2054.861572
min         30.926200
25%        744.254406
50%       1523.556702
75%       2785.461738
max      13871.198616
Name: revenue, dtype: float64


In [42]:
# Select features for training
# We want to predict revenue, so we'll use features that don't include revenue itself
feature_columns = [
    'base_price',
    'offered_discount',
    'category',
    'past_sales_volume',
    'price_elasticity_estimate',
    'month',
    'day_of_week',
    'is_weekend',
    'sales_volume_percentile'
]

# Target variable: revenue (what we want to maximize)
target_column = 'revenue'

# Filter to purchased items only (positive revenue)
df_train = df[df['purchased'] == 1].copy()

print(f"📊 Training data shape: {df_train.shape}")
print(f"📊 Target variable stats:\n{df_train[target_column].describe()}")

# Encode categorical features
category_encoder = LabelEncoder()
df_train['category_encoded'] = category_encoder.fit_transform(df_train['category'])

# Update feature columns to use encoded category
feature_columns_encoded = [
    'base_price',
    'offered_discount',
    'category_encoded',
    'past_sales_volume',
    'price_elasticity_estimate',
    'month',
    'day_of_week',
    'is_weekend',
    'sales_volume_percentile'
]

# Prepare X and y
X = df_train[feature_columns_encoded].copy()
y = df_train[target_column].copy()

print(f"\n✅ Features prepared:")
print(f"   Feature shape: {X.shape}")
print(f"   Target shape: {y.shape}")
print(f"\n📋 Feature columns: {feature_columns_encoded}")
print(f"\n📊 Feature statistics:")
print(X.describe())


📊 Training data shape: (3138, 25)
📊 Target variable stats:
count     3138.000000
mean      2152.531580
std       2054.861572
min         30.926200
25%        744.254406
50%       1523.556702
75%       2785.461738
max      13871.198616
Name: revenue, dtype: float64

✅ Features prepared:
   Feature shape: (3138, 9)
   Target shape: (3138,)

📋 Feature columns: ['base_price', 'offered_discount', 'category_encoded', 'past_sales_volume', 'price_elasticity_estimate', 'month', 'day_of_week', 'is_weekend', 'sales_volume_percentile']

📊 Feature statistics:
        base_price  offered_discount  category_encoded  past_sales_volume  \
count  3138.000000       3138.000000       3138.000000        3138.000000   
mean    221.325319         14.843811          3.497769          33.873805   
std     150.448485          8.601924          2.305244          22.572054   
min      40.400000          0.000000          0.000000           1.000000   
25%     116.265000          7.370000          1.000000        

## 5. Split Dataset (80/20)


In [43]:
# Split dataset
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, shuffle=True
)

print(f"✅ Dataset split:")
print(f"   Training set: {X_train.shape[0]} samples")
print(f"   Test set: {X_test.shape[0]} samples")
print(f"   Split ratio: {X_train.shape[0] / (X_train.shape[0] + X_test.shape[0]):.1%}")


✅ Dataset split:
   Training set: 2510 samples
   Test set: 628 samples
   Split ratio: 80.0%


## 6. Scale Features


In [44]:
# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("✅ Features scaled successfully")
print(f"   Scaled training shape: {X_train_scaled.shape}")
print(f"   Scaled test shape: {X_test_scaled.shape}")


✅ Features scaled successfully
   Scaled training shape: (2510, 9)
   Scaled test shape: (628, 9)


## 7. Train Multiple Models


In [45]:
print("🤖 Training multiple regression models...\n")

models = {}
results = {}

# 1. Random Forest Regressor
print("1️⃣ Training RandomForestRegressor...")
rf_model = RandomForestRegressor(
    n_estimators=100,
    max_depth=15,
    min_samples_split=5,
    min_samples_leaf=2,
    random_state=42,
    n_jobs=-1
)
rf_model.fit(X_train_scaled, y_train)
models['RandomForest'] = rf_model

rf_pred = rf_model.predict(X_test_scaled)
rf_r2 = r2_score(y_test, rf_pred)
rf_rmse = np.sqrt(mean_squared_error(y_test, rf_pred))
rf_mae = mean_absolute_error(y_test, rf_pred)

results['RandomForest'] = {
    'r2': rf_r2,
    'rmse': rf_rmse,
    'mae': rf_mae
}

print(f"   ✅ R² Score: {rf_r2:.4f}")
print(f"   ✅ RMSE: {rf_rmse:.4f}")
print(f"   ✅ MAE: {rf_mae:.4f}\n")

# 2. XGBoost Regressor
print("2️⃣ Training XGBoostRegressor...")
xgb_model = xgb.XGBRegressor(
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
    random_state=42,
    n_jobs=-1
)
xgb_model.fit(X_train_scaled, y_train)
models['XGBoost'] = xgb_model

xgb_pred = xgb_model.predict(X_test_scaled)
xgb_r2 = r2_score(y_test, xgb_pred)
xgb_rmse = np.sqrt(mean_squared_error(y_test, xgb_pred))
xgb_mae = mean_absolute_error(y_test, xgb_pred)

results['XGBoost'] = {
    'r2': xgb_r2,
    'rmse': xgb_rmse,
    'mae': xgb_mae
}

print(f"   ✅ R² Score: {xgb_r2:.4f}")
print(f"   ✅ RMSE: {xgb_rmse:.4f}")
print(f"   ✅ MAE: {xgb_mae:.4f}\n")

# 3. Linear Regression
print("3️⃣ Training LinearRegression...")
lr_model = LinearRegression()
lr_model.fit(X_train_scaled, y_train)
models['LinearRegression'] = lr_model

lr_pred = lr_model.predict(X_test_scaled)
lr_r2 = r2_score(y_test, lr_pred)
lr_rmse = np.sqrt(mean_squared_error(y_test, lr_pred))
lr_mae = mean_absolute_error(y_test, lr_pred)

results['LinearRegression'] = {
    'r2': lr_r2,
    'rmse': lr_rmse,
    'mae': lr_mae
}

print(f"   ✅ R² Score: {lr_r2:.4f}")
print(f"   ✅ RMSE: {lr_rmse:.4f}")
print(f"   ✅ MAE: {lr_mae:.4f}\n")

print("✅ All models trained successfully!")


🤖 Training multiple regression models...

1️⃣ Training RandomForestRegressor...
   ✅ R² Score: 0.8570
   ✅ RMSE: 777.4238
   ✅ MAE: 486.7777

2️⃣ Training XGBoostRegressor...
   ✅ R² Score: 0.8431
   ✅ RMSE: 814.4095
   ✅ MAE: 489.8282

3️⃣ Training LinearRegression...
   ✅ R² Score: 0.7240
   ✅ RMSE: 1079.9885
   ✅ MAE: 763.3328

✅ All models trained successfully!


In [46]:
# Compare models
comparison_df = pd.DataFrame(results).T
comparison_df = comparison_df.sort_values('r2', ascending=False)

print("📊 Model Comparison:")
print("=" * 60)
print(comparison_df.to_string())
print("=" * 60)

# Select best model based on R² and RMSE
# Higher R² and lower RMSE is better
comparison_df['score'] = comparison_df['r2'] - (comparison_df['rmse'] / comparison_df['rmse'].max())
best_model_name = comparison_df.sort_values('score', ascending=False).index[0]
best_model = models[best_model_name]

print(f"\n🏆 Best Model: {best_model_name}")
print(f"   R² Score: {results[best_model_name]['r2']:.4f}")
print(f"   RMSE: {results[best_model_name]['rmse']:.4f}")
print(f"   MAE: {results[best_model_name]['mae']:.4f}")

# Store for later use
selected_model = best_model
selected_model_name = best_model_name


📊 Model Comparison:
                        r2         rmse         mae
RandomForest      0.857006   777.423808  486.777732
XGBoost           0.843077   814.409471  489.828220
LinearRegression  0.724044  1079.988481  763.332846

🏆 Best Model: RandomForest
   R² Score: 0.8570
   RMSE: 777.4238
   MAE: 486.7777


In [47]:
def predict_optimal_discount(product_id, base_price, category, past_sales_volume, 
                             model, scaler, category_encoder, training_df=None,
                             month=None, day_of_week=None, is_weekend=None):
    """
    Predict optimal discount (0-30%) for a product that maximizes revenue.
    
    Parameters:
    -----------
    product_id : str
        Product identifier
    base_price : float
        Base price of the product
    category : str
        Product category
    past_sales_volume : float
        Historical sales volume for the product
    model : sklearn model
        Trained regression model
    scaler : sklearn scaler
        Fitted StandardScaler
    category_encoder : sklearn LabelEncoder
        Fitted LabelEncoder for categories
    training_df : pd.DataFrame, optional
        Training dataframe to calculate elasticity patterns
    month : int, optional
        Month (1-12), defaults to current month
    day_of_week : int, optional
        Day of week (0-6), defaults to current day
    is_weekend : int, optional
        1 if weekend, 0 otherwise, defaults to current day
    
    Returns:
    --------
    dict : {
        "product": str,
        "optimal_discount": float,
        "expected_revenue": float,
        "final_selling_price": float,
        "confidence": str
    }
    """
    # Default to current date if not provided
    if month is None:
        month = datetime.now().month
    if day_of_week is None:
        day_of_week = datetime.now().weekday()
    if is_weekend is None:
        is_weekend = 1 if datetime.now().weekday() >= 5 else 0
    
    # Encode category
    try:
        category_encoded = category_encoder.transform([category])[0]
    except ValueError:
        # If category not seen, use most common category
        category_encoded = category_encoder.transform([category_encoder.classes_[0]])[0]
    
    # Calculate sales volume percentile
    sales_volume_percentile = 0  # Default
    
    # Learn elasticity patterns from training data if available
    if training_df is not None:
        # Calculate average elasticity by category from training data
        category_elasticity = training_df.groupby('category')['price_elasticity_estimate'].mean()
        base_elasticity = category_elasticity.get(category, -1.5)
        
        # Calculate average quantity at 0% discount for this category
        base_quantity = training_df[
            (training_df['category'] == category) & 
            (training_df['offered_discount'] < 1)
        ]['quantity_ordered'].mean()
        if pd.isna(base_quantity) or base_quantity == 0:
            base_quantity = training_df[training_df['category'] == category]['quantity_ordered'].mean()
        if pd.isna(base_quantity) or base_quantity == 0:
            base_quantity = 5.0  # Default fallback
    else:
        base_elasticity = -1.5
        base_quantity = 5.0
    
    # Simulate different discount percentages (0-30%)
    discount_range = np.arange(0, 31, 0.5)  # 0 to 30% in 0.5% increments
    revenue_predictions = []
    
    for discount_pct in discount_range:
        # Calculate discounted price
        discounted_price = base_price * (1 - discount_pct / 100)
        
        # Estimate how discount affects quantity (price elasticity)
        # Higher discount = higher quantity, but with diminishing returns
        price_change_pct = discount_pct / 100
        
        # Price elasticity: % change in quantity / % change in price
        # If elasticity = -2, a 1% price drop leads to 2% quantity increase
        # Convert elasticity to positive for calculation (elasticity is negative)
        elasticity_magnitude = abs(base_elasticity)
        
        # Quantity increase from price elasticity
        # For every 1% price drop, quantity increases by elasticity_magnitude%
        quantity_elasticity_boost = 1 + (elasticity_magnitude * price_change_pct)
        
        # Additional conversion boost (higher discount = more likely to purchase)
        # Higher discounts increase conversion probability
        conversion_boost = 1 + (price_change_pct * 0.5)  # 10% discount = 5% conversion boost
        
        # Estimate expected quantity with discount
        expected_quantity = base_quantity * quantity_elasticity_boost * conversion_boost
        
        # Estimate price elasticity for the feature vector (for model input)
        # Elasticity becomes more negative (more elastic) with higher discounts
        elasticity = base_elasticity * (1 + price_change_pct * 0.2)
        
        # Prepare feature vector
        features = np.array([[
            base_price,
            discount_pct,
            category_encoded,
            past_sales_volume,
            elasticity,
            month,
            day_of_week,
            is_weekend,
            sales_volume_percentile
        ]])
        
        # Scale features
        features_scaled = scaler.transform(features)
        
        # Predict revenue from model (model was trained on actual revenue = price × quantity)
        predicted_revenue_model = model.predict(features_scaled)[0]
        
        # Calculate revenue directly from expected quantity and discounted price
        # This properly accounts for elasticity: Revenue = Price × Quantity
        # Where Quantity increases with discount based on elasticity
        expected_revenue_direct = discounted_price * expected_quantity
        
        # The model learned from historical data, but might be conservative
        # Use the direct calculation which properly models elasticity
        # But blend with model prediction for robustness
        final_expected_revenue = 0.7 * expected_revenue_direct + 0.3 * predicted_revenue_model
        
        # Only consider positive revenue
        if final_expected_revenue > 0:
            revenue_predictions.append({
                'discount': discount_pct,
                'revenue': final_expected_revenue,
                'discounted_price': discounted_price,
                'expected_quantity': expected_quantity
            })
    
    if not revenue_predictions:
        # Fallback: return 0% discount
        optimal_discount = 0.0
        expected_revenue = base_price * base_quantity
        discounted_price = base_price
        confidence = "Low"
    else:
        # Find discount that maximizes revenue
        revenue_df = pd.DataFrame(revenue_predictions)
        optimal_idx = revenue_df['revenue'].idxmax()
        optimal_discount = revenue_df.loc[optimal_idx, 'discount']
        expected_revenue = revenue_df.loc[optimal_idx, 'revenue']
        discounted_price = revenue_df.loc[optimal_idx, 'discounted_price']
        
        # Calculate confidence based on revenue difference
        max_revenue = revenue_df['revenue'].max()
        second_max_revenue = revenue_df.nlargest(2, 'revenue')['revenue'].iloc[-1] if len(revenue_df) > 1 else max_revenue
        revenue_diff_pct = ((max_revenue - second_max_revenue) / (second_max_revenue + 1e-6)) * 100
        
        if revenue_diff_pct > 10:
            confidence = "High"
        elif revenue_diff_pct > 5:
            confidence = "Medium"
        else:
            confidence = "Low"
    
    # Get product name (simplified - in practice, query from database)
    product_name = f"Product {product_id}"
    
    return {
        "product": product_name,
        "optimal_discount": round(optimal_discount, 2),
        "expected_revenue": round(expected_revenue, 2),
        "final_selling_price": round(discounted_price, 2),
        "confidence": confidence
    }

print("✅ Optimal discount prediction function created (improved with elasticity patterns)")


✅ Optimal discount prediction function created (improved with elasticity patterns)


## 10.5. Test Discount Calculation Logic


In [48]:
# Quick test to verify discount calculation logic
print("🧪 Testing discount calculation logic...\n")

# Test with a sample product
test_base_price = 100.0
test_base_quantity = 5.0
test_elasticity = -2.0  # Elastic price: 1% price drop → 2% quantity increase

print(f"Test parameters:")
print(f"  Base Price: ₹{test_base_price}")
print(f"  Base Quantity: {test_base_quantity} units")
print(f"  Elasticity: {test_elasticity} (elastic product)\n")

for discount in [0, 5, 10, 15, 20, 25, 30]:
    price_change = discount / 100
    discounted_price = test_base_price * (1 - price_change)
    
    # Quantity increase from elasticity
    quantity_boost = 1 + (abs(test_elasticity) * price_change)
    conversion_boost = 1 + (price_change * 0.5)
    expected_quantity = test_base_quantity * quantity_boost * conversion_boost
    
    revenue = discounted_price * expected_quantity
    revenue_change_pct = ((revenue - (test_base_price * test_base_quantity)) / (test_base_price * test_base_quantity)) * 100
    
    print(f"  {discount:2d}% discount: Price=₹{discounted_price:6.2f}, Qty={expected_quantity:5.2f}, Revenue=₹{revenue:7.2f} ({revenue_change_pct:+.1f}%)")

print("\n✅ Discount calculation test completed")
print("   Note: With elasticity > 1, discounts should increase revenue!")


🧪 Testing discount calculation logic...

Test parameters:
  Base Price: ₹100.0
  Base Quantity: 5.0 units
  Elasticity: -2.0 (elastic product)

   0% discount: Price=₹100.00, Qty= 5.00, Revenue=₹ 500.00 (+0.0%)
   5% discount: Price=₹ 95.00, Qty= 5.64, Revenue=₹ 535.56 (+7.1%)
  10% discount: Price=₹ 90.00, Qty= 6.30, Revenue=₹ 567.00 (+13.4%)
  15% discount: Price=₹ 85.00, Qty= 6.99, Revenue=₹ 593.94 (+18.8%)
  20% discount: Price=₹ 80.00, Qty= 7.70, Revenue=₹ 616.00 (+23.2%)
  25% discount: Price=₹ 75.00, Qty= 8.44, Revenue=₹ 632.81 (+26.6%)
  30% discount: Price=₹ 70.00, Qty= 9.20, Revenue=₹ 644.00 (+28.8%)

✅ Discount calculation test completed
   Note: With elasticity > 1, discounts should increase revenue!


In [49]:
# Preserve model training results before demo
model_results = results.copy()  # Save model results dictionary
print("✅ Model results preserved for validation summary")


✅ Model results preserved for validation summary


In [50]:
# Create models directory if it doesn't exist
models_dir = '../ml_models'
os.makedirs(models_dir, exist_ok=True)

# Save model artifacts
model_path = os.path.join(models_dir, 'dynamic_pricing_model.pkl')
scaler_path = os.path.join(models_dir, 'input_scaler.pkl')
encoder_path = os.path.join(models_dir, 'product_encoder.pkl')

joblib.dump(selected_model, model_path)
joblib.dump(scaler, scaler_path)
joblib.dump(category_encoder, encoder_path)

print("✅ Model artifacts saved:")
print(f"   Model: {model_path}")
print(f"   Scaler: {scaler_path}")
print(f"   Encoder: {encoder_path}")
print(f"\n📋 Model info:")
print(f"   Model type: {selected_model_name}")
print(f"   Features: {len(feature_columns_encoded)}")
print(f"   Feature names: {feature_columns_encoded}")


✅ Model artifacts saved:
   Model: ../ml_models\dynamic_pricing_model.pkl
   Scaler: ../ml_models\input_scaler.pkl
   Encoder: ../ml_models\product_encoder.pkl

📋 Model info:
   Model type: RandomForest
   Features: 9
   Feature names: ['base_price', 'offered_discount', 'category_encoded', 'past_sales_volume', 'price_elasticity_estimate', 'month', 'day_of_week', 'is_weekend', 'sales_volume_percentile']


## 11. Demo: Test Optimal Discount Prediction


In [51]:
print("🎯 Demo: Optimal Discount Predictions for Sample Products\n")
print("=" * 80)

# Test with 5 different products
demo_products = [
    {
        'product_id': 'PROD_Vegetables_1',
        'product_name': 'Organic Tomatoes',
        'base_price': 80.0,
        'category': 'Vegetables',
        'past_sales_volume': 150
    },
    {
        'product_id': 'PROD_Fruits_1',
        'product_name': 'Fresh Mangoes',
        'base_price': 120.0,
        'category': 'Fruits',
        'past_sales_volume': 200
    },
    {
        'product_id': 'PROD_Grains_1',
        'product_name': 'Toor Dal',
        'base_price': 120.0,
        'category': 'Grains',
        'past_sales_volume': 300
    },
    {
        'product_id': 'PROD_Spices_1',
        'product_name': 'Turmeric Powder',
        'base_price': 250.0,
        'category': 'Spices',
        'past_sales_volume': 100
    },
    {
        'product_id': 'PROD_Dairy_1',
        'product_name': 'Fresh Milk',
        'base_price': 60.0,
        'category': 'Dairy',
        'past_sales_volume': 500
    }
]

results = []
for product in demo_products:
    result = predict_optimal_discount(
        product_id=product['product_id'],
        base_price=product['base_price'],
        category=product['category'],
        past_sales_volume=product['past_sales_volume'],
        model=selected_model,
        scaler=scaler,
        category_encoder=category_encoder,
        training_df=df_train  # Pass training data to learn elasticity patterns
    )
    
    # Update with actual product name
    result['product'] = product['product_name']
    
    results.append({
        **product,
        **result
    })
    
    print(f"\n📦 {product['product_name']}")
    print(f"   Base Price: ₹{product['base_price']:.2f}")
    print(f"   Category: {product['category']}")
    print(f"   Past Sales Volume: {product['past_sales_volume']}")
    print(f"   ─────────────────────────────────────────")
    print(f"   💡 Optimal Discount: {result['optimal_discount']:.1f}%")
    print(f"   💰 Final Selling Price: ₹{result['final_selling_price']:.2f}")
    print(f"   📈 Expected Revenue: ₹{result['expected_revenue']:.2f}")
    print(f"   🎯 Confidence: {result['confidence']}")

print("\n" + "=" * 80)
print("\n✅ Demo completed successfully!")


🎯 Demo: Optimal Discount Predictions for Sample Products


📦 Organic Tomatoes
   Base Price: ₹80.00
   Category: Vegetables
   Past Sales Volume: 150
   ─────────────────────────────────────────
   💡 Optimal Discount: 30.5%
   💰 Final Selling Price: ₹55.60
   📈 Expected Revenue: ₹873.87
   🎯 Confidence: Low

📦 Fresh Mangoes
   Base Price: ₹120.00
   Category: Fruits
   Past Sales Volume: 200
   ─────────────────────────────────────────
   💡 Optimal Discount: 30.5%
   💰 Final Selling Price: ₹83.40
   📈 Expected Revenue: ₹1447.79
   🎯 Confidence: Low

📦 Toor Dal
   Base Price: ₹120.00
   Category: Grains
   Past Sales Volume: 300
   ─────────────────────────────────────────
   💡 Optimal Discount: 24.5%
   💰 Final Selling Price: ₹90.60
   📈 Expected Revenue: ₹1443.54
   🎯 Confidence: Low

📦 Turmeric Powder
   Base Price: ₹250.00
   Category: Spices
   Past Sales Volume: 100
   ─────────────────────────────────────────
   💡 Optimal Discount: 30.5%
   💰 Final Selling Price: ₹173.75
   📈 Exp

## 12. Error Handling: Unseen Products


In [52]:
# Fix: Restore model results if they were overwritten by demo
if 'model_results' in globals():
    results = model_results.copy()
    print("✅ Model results restored for validation summary")
else:
    print("⚠️  Note: Run cell 21 before the demo to preserve model results")
    print("   If you see this error, re-run cells 14-16 (model training) first")


✅ Model results restored for validation summary


In [53]:
# Test with unseen category
print("🧪 Testing error handling for unseen products...\n")

# Test with unseen category
try:
    result = predict_optimal_discount(
        product_id='PROD_UNSEEN_1',
        base_price=100.0,
        category='UnseenCategory',  # Category not in training data
        past_sales_volume=50,
        model=selected_model,
        scaler=scaler,
        category_encoder=category_encoder,
        training_df=df_train
    )
    print("✅ Unseen category handled gracefully")
    print(f"   Result: {result}")
except Exception as e:
    print(f"❌ Error with unseen category: {e}")

# Test with category average (fallback)
print("\n📊 Category average discount analysis:")
category_avg_discounts = df_train.groupby('category')['offered_discount'].mean()
print(category_avg_discounts)

print("\n✅ Error handling verified!")


🧪 Testing error handling for unseen products...

✅ Unseen category handled gracefully
   Result: {'product': 'Product PROD_UNSEEN_1', 'optimal_discount': np.float64(0.0), 'expected_revenue': np.float64(559.34), 'final_selling_price': np.float64(100.0), 'confidence': 'Low'}

📊 Category average discount analysis:
category
Dairy         15.010808
Fruits        14.732735
Grains        14.392474
Nuts          14.614661
Oil           14.990120
Pulses        15.014667
Spices        15.416559
Vegetables    14.503005
Name: offered_discount, dtype: float64

✅ Error handling verified!


## 13. Model Validation Summary


In [54]:
print("📊 Model Validation Summary")
print("=" * 80)

print(f"\n🏆 Selected Model: {selected_model_name}")
print(f"   R² Score: {results[selected_model_name]['r2']:.4f}")
print(f"   RMSE: {results[selected_model_name]['rmse']:.4f}")
print(f"   MAE: {results[selected_model_name]['mae']:.4f}")

print(f"\n📋 Training Data:")
print(f"   Total samples: {len(df_train)}")
print(f"   Training set: {len(X_train)}")
print(f"   Test set: {len(X_test)}")

print(f"\n🔧 Features Used ({len(feature_columns_encoded)}):")
for i, feat in enumerate(feature_columns_encoded, 1):
    print(f"   {i}. {feat}")

print(f"\n💾 Saved Artifacts:")
print(f"   ✓ dynamic_pricing_model.pkl")
print(f"   ✓ input_scaler.pkl")
print(f"   ✓ product_encoder.pkl")

print(f"\n✅ Model is ready for deployment!")
print(f"   Use predict_optimal_discount() function to get recommendations")


📊 Model Validation Summary

🏆 Selected Model: RandomForest
   R² Score: 0.8570
   RMSE: 777.4238
   MAE: 486.7777

📋 Training Data:
   Total samples: 3138
   Training set: 2510
   Test set: 628

🔧 Features Used (9):
   1. base_price
   2. offered_discount
   3. category_encoded
   4. past_sales_volume
   5. price_elasticity_estimate
   6. month
   7. day_of_week
   8. is_weekend
   9. sales_volume_percentile

💾 Saved Artifacts:
   ✓ dynamic_pricing_model.pkl
   ✓ input_scaler.pkl
   ✓ product_encoder.pkl

✅ Model is ready for deployment!
   Use predict_optimal_discount() function to get recommendations
